In [21]:
import joblib
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from numpy import sort
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
%matplotlib

###########fix random seed for reproducability##########

import sys
sys.path.append("../../../lib")
csv = "../../../data/database.csv"

def get_degradation_type(x):
    if x>50:
        return 1
    return 2


seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv(csv,encoding="gbk", index_col=0)
raw_data_all=fdata.loc[0:895,]
drop = ["SMILES", "RDKit_Mol_Class","Clusters_5","Clusters_10"]
raw_data = raw_data_all.drop(columns=drop)
raw_data["lei"]=raw_data["Degradation(%)"].apply(get_degradation_type)
##########handling missing values##########
median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)

###########data standardization##########
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)#即简单实现标准化
###########train test splitting##########
parame=standardized_data.iloc[:,0:163]
degradation=standardized_data.iloc[:,164]
X=parame.values.astype(np.float32)
y=degradation.values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=751)


import xgboost as xgb
from sklearn.metrics import roc_curve, auc
model_XGboostRegressor=xgb.XGBRegressor()
model = joblib.load('filepath.xgb')

y_real_degradation = y_test*np.std(data,axis=0)[164]+np.mean(data,axis=0)[164]
csvsave=pd.DataFrame(y_real_degradation)
csvsave.columns = ['real']

for nmax_features in range(5,50,5):
    selection = SelectFromModel(model, max_features=nmax_features,threshold=-1, prefit=True)
    select_X_train = selection.transform(X_train)
    select_X_test  = selection.transform(X_test)
    # train model
    param = {
        'learning_rate':[0.1],
        'n_estimators':[8000],
        'max_depth':[9],  
        'objective':['reg:squarederror'],
        'subsample':[0.8],
        'lambda':[0.1],
        'alpha':[0.1]
        }
    grid = GridSearchCV(model_XGboostRegressor,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    selection_model = grid.best_estimator_
    #selection_model = XGBClassifier(reg_alpha = 0.1,reg_lambda=0.1,learning_rate=0.01, max_depth= 5, n_estimators=200, objective='reg:squarederror', subsample=0.5)
    selection_model.fit(select_X_train, y_train)  
    #eval model
    result = selection_model.predict(select_X_test)
    x_prediction_degradation_ann = result*np.std(data,axis=0)[164]+np.mean(data,axis=0)[164]
    csvsave['%d'%nmax_features]=x_prediction_degradation_ann
    #y_real_degradation = y_test*np.std(data,axis=0)[164]+np.mean(data,axis=0)[164]
    x_prediction_degradation_ann_series = pd.Series(x_prediction_degradation_ann)
    y_real_degradation_series = pd.Series(y_real_degradation)
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_degradation_ann_series.corr(y_real_degradation_series), 4)
    rmse_val = rmse(x_prediction_degradation_ann,y_real_degradation)
    print("n=%d, Accuracy: %.2f%%" % (select_X_train.shape[1], corr_ann*100.0))
csvsave.to_csv('test.csv')


Using matplotlib backend: Qt5Agg
n=5, Accuracy: 73.21%
n=10, Accuracy: 63.99%
n=15, Accuracy: 66.06%
n=20, Accuracy: 49.27%
n=25, Accuracy: 81.83%
n=30, Accuracy: 82.79%


KeyboardInterrupt: 

In [35]:
import joblib
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from numpy import sort
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
%matplotlib

###########fix random seed for reproducability##########

import sys
sys.path.append("../../../lib")
csv = "../../../data/database.csv"

def get_degradation_type(x):
    if x>50:
        return 1
    return 2


seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv(csv,encoding="gbk", index_col=0)
raw_data_all=fdata.loc[0:895,]
drop = ["SMILES", "RDKit_Mol_Class","Clusters_5","Clusters_10"]
raw_data = raw_data_all.drop(columns=drop)
raw_data["lei"]=raw_data["Degradation(%)"].apply(get_degradation_type)
##########handling missing values##########
median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)

###########data standardization##########
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)#即简单实现标准化
###########train test splitting##########
parame=standardized_data.iloc[:,0:163]
degradation=standardized_data.iloc[:,164]
degradation=raw_data["lei"]
X=parame.values.astype(np.float32)
#y=degradation.values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, degradation, test_size=.15, random_state=751)

###########defining a wrapper function for later call from each machine learning algorithms##########
def try_different_method(model):
    model.fit(X_train,y_train)
    score = model.score(X_test, y_test)
    result = model.predict(X_test)
    x_prediction_degradation_ann=result*np.std(data,axis=0)[164]+np.mean(data,axis=0)[164]
    y_real_degradation=y_test*np.std(data,axis=0)[164]+np.mean(data,axis=0)[164]
    print(x_prediction_degradation_ann)
    x_prediction_degradation_ann_series=pd.Series(x_prediction_degradation_ann)
    y_real_degradation_series=pd.Series(y_real_degradation)
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_degradation_ann_series.corr(y_real_degradation_series), 4)
    rmse_val= rmse(x_prediction_degradation_ann,y_real_degradation)
    print(rmse_val)
    print(corr_ann)
    print(y_real_degradation)
    ###########generating a figure##########
    x_y_x=np.arange(0,100,10)
    x_y_y=np.arange(0,100,10)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x_prediction_degradation_ann,y_real_degradation,color='red',label='Artificial Neural Network')
    plt.legend()
    ax.plot(x_y_x,x_y_y)
    plt.xlabel(u"Predicted_Degradable_Conductivity %")
    plt.ylabel(u"Real_Degradable_Conductivity %")
    plt.show()

from xgboost import XGBClassifier
model_XGboostRegressor=XGBClassifier()

Using matplotlib backend: Qt5Agg


In [37]:
seed=1
np.random.seed(seed)
####XGboost####
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]
)

###########defining the parameters dictionary##########
param = {
        'learning_rate':[0.01,0.1,1],
        'n_estimators':[100,200,400,1000,2000,4000,8000],
        'max_depth':[5,7,9,11],  
        'objective':['binary:logistic','binary:logitraw'],
        'subsample':[0.5,0.6,0.7,0.8,0.9],
        'lambda':[0.1],
        'alpha':[0.1]
       }
grid = GridSearchCV(model_XGboostRegressor,param_grid=param,cv=5)
grid.fit(X_train,y_train)
print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_) 
best_model=grid.best_estimator_

algorithm_name='XGBoost Regressor'
try_different_method(best_model)

C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:48:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[21:48:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1

[21:48:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[21:48:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:48:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1

...

[00:02:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:02:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:02:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:02:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1

[00:03:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:03:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:03:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:03:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[00:04:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:04:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:04:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:04:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1

[00:05:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:05:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:05:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:05:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

[00:06:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:07:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:07:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:07:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1

In [50]:
import joblib
joblib.dump(best_model, filename='XGB_Classifier.xgb')

['XGB_Classifier.xgb']

In [49]:
from sklearn.metrics import roc_curve, auc
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score
gbc = best_model
gbc.fit(X_train, y_train)
resu = gbc.predict(X_test) #进行预测
y_pred_gbc = gbc.predict_proba(X_test)[:,1] ###这玩意就是预测概率的
fpr, tpr, threshold = roc_curve(y_test, y_pred_gbc,  pos_label=2) ###画图的时候要用预测的概率，而不是你的预测的值
rocauc=roc_auc_score(y_test, y_pred_gbc)
plt.plot(fpr, tpr, 'b', label='AUC = %0.4f' % rocauc)#生成ROC曲线
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.ylabel('真正率')
plt.xlabel('假正率')
plt.show()

C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:31:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 20551 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 27491 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 29575 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\matplotlib\backends\backend_agg.py:180: RuntimeWarning: Glyph 20551 missing from current font.
  font.set_text(s, 0, flags=flags)
C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\matplotlib\backends\backend_agg.py:180: RuntimeWarning: Glyph 27491 missing from current font.
  font.set_text(s, 0, flags=flags)
C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\matplotlib\backends\backen

In [ ]:
fig = plt.figure() 
ax = fig.add_subplot(111)
ax.scatter(x_time1,predict_result_list1,color='red',label='Predicted Conductivity remaining (%)')
ax.plot(x_time1,data.iloc[database_list[0],19:].values,color='orange')
ax.set_xlabel(u"Time(h)")
ax.set_ylabel(u"Conductivity remaining(%)")

In [67]:
import joblib
model1 = joblib.load('XGB_Classifier.xgb')
model = joblib.load('XGB_Classifier.xgb')
selection = SelectFromModel(model, max_features=42,threshold=-1, prefit=True)
X_transform = selection.transform(X)
X_transform.shape
csvsave=pd.DataFrame()
for nmax_features in range(5,50,5):
    selection = SelectFromModel(model1, max_features=nmax_features,threshold=-1, prefit=True)
    select_X_train = selection.transform(X_train)
    select_X_test  = selection.transform(X_test)
    # train model
    model.fit(select_X_train, y_train)
    y_pred_gbc = model.predict_proba(select_X_test)[:,1] ###这玩意就是预测概率的
    fpr, tpr, threshold = roc_curve(y_test, y_pred_gbc,  pos_label=2) ###画图的时候要用预测的概率，而不是你的预测的值
    rocauc=roc_auc_score(y_test, y_pred_gbc)
    print("n=%d, Accuracy: %.2f%%" % (select_X_train.shape[1], rocauc*100.0))
    csvsave=pd.concat([csvsave,pd.DataFrame(fpr)],axis=1)
    csvsave=pd.concat([csvsave,pd.DataFrame(tpr)],axis=1)
csvsave.to_csv('1.csv')

C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:07:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
n=5, Accuracy: 90.73%


C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:07:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
n=10, Accuracy: 93.51%


C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

n=35, Accuracy: 97.94%
[21:07:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


n=40, Accuracy: 98.40%
[21:07:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


n=45, Accuracy: 98.89%


In [66]:
a=[1,2]
csvsave=pd.DataFrame()
print(csvsave)
b=[1,2,3]
csvsave=pd.concat([csvsave,pd.DataFrame(a)],axis=1)
csvsave=pd.concat([csvsave,pd.DataFrame(b)],axis=1)
print(csvsave)

Empty DataFrame
Columns: []
Index: []
     0  0
0  1.0  1
1  2.0  2
2  NaN  3


In [68]:
print(y_pred_gbc )

[5.5182701e-01 9.9955803e-01 7.3886849e-04 1.9389817e-01 9.9838781e-01
 9.8985124e-01 9.0517533e-01 9.9218893e-01 9.9256259e-01 9.5122266e-01
 9.9881870e-01 1.2178501e-01 9.9967885e-01 9.9881101e-01 9.9758053e-01
 9.9955815e-01 1.8051818e-03 9.9969089e-01 9.4626385e-01 9.9900752e-01
 9.6805197e-01 9.9960321e-01 9.9950123e-01 9.9986565e-01 7.4178082e-01
 9.9977392e-01 9.9984455e-01 9.6989787e-01 9.9696547e-01 1.2079746e-02
 9.9948883e-01 9.9959701e-01 9.9981886e-01 9.9911457e-01 9.9907070e-01
 9.9097532e-01 9.9660885e-01 9.9905318e-01 3.0842133e-02 9.9952996e-01
 8.4116182e-04 9.9491173e-01 9.9976176e-01 1.7061342e-03 9.8084199e-01
 9.9741495e-01 9.9940205e-01 9.9954176e-01 9.6899688e-01 9.9608648e-01
 9.9422014e-01 9.9902213e-01 9.9927622e-01 9.9840134e-01 1.2193220e-02
 9.9983156e-01 9.9863356e-01 9.9444121e-01 9.9842453e-01 4.5052073e-03
 9.9778807e-01 2.1388056e-03 9.9032158e-01 9.5663762e-01 9.9873739e-01
 9.9990320e-01 9.9967110e-01 9.9902463e-01 1.0877869e-03 1.7108571e-02
 9.934

In [41]:
y_true=np.array(y_test,dtype=binary)
y_result=np.array(result,dtype=np.float32)

NameError: name 'binary' is not defined

In [73]:
from sklearn.metrics import roc_curve, auc
from xgboost.sklearn import XGBClassifier

gbc = XGBClassifier()
gbc.fit(select_X_train, y_train)
resu = gbc.predict(select_X_test) #进行预测
y_pred_gbc = gbc.predict_proba(select_X_test)[:,1] ###这玩意就是预测概率的
fpr, tpr, threshold = roc_curve(y_test, y_pred_gbc) ###画图的时候要用预测的概率，而不是你的预测的值
rocauc= 
plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % rocauc)#生成ROC曲线
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('真正率')
plt.xlabel('假正率')
plt.show()

[17:03:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ValueError: continuous format is not supported

In [35]:
from sklearn.metrics import explained_variance_score
score = explained_variance_score(result,y_test)
print(score)

0.05005526542663574


In [45]:
y = np.array([ 1,  2, 3, 1])
print(y)
print(type_of_target(y))

[1 2 3 1]
multiclass


In [40]:
print(type_of_target(y))

binary


In [75]:
selection = SelectFromModel(model1, max_features=100,threshold=-1, prefit=True)
select_X_train = selection.transform(X_train)
select_X_test  = selection.transform(X_test)
# train model
model.fit(select_X_train, y_train)
resu = model.predict(select_X_test)
mean_fpr = np.linspace(0, 1, 100)
tprs = []
interp_tpr = np.interp(mean_fpr, y_test-1, resu)
interp_tpr[0] = 0.0
tprs.append(interp_tpr)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

plt.plot(mean_fpr, mean_tpr,lw=2, alpha=.8)
plt.fill_between(mean_fpr, tprs_upper, tprs_lower, alpha=.2)
plt.show()

C:\ProgramData\Anaconda3\envs\tf21\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:21:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [71]:
print( y_test-1)
print( y_pred_gbc)

340    0
140    1
528    0
5      1
833    1
      ..
371    1
644    1
692    1
829    1
566    0
Name: lei, Length: 135, dtype: int64
[5.5182701e-01 9.9955803e-01 7.3886849e-04 1.9389817e-01 9.9838781e-01
 9.8985124e-01 9.0517533e-01 9.9218893e-01 9.9256259e-01 9.5122266e-01
 9.9881870e-01 1.2178501e-01 9.9967885e-01 9.9881101e-01 9.9758053e-01
 9.9955815e-01 1.8051818e-03 9.9969089e-01 9.4626385e-01 9.9900752e-01
 9.6805197e-01 9.9960321e-01 9.9950123e-01 9.9986565e-01 7.4178082e-01
 9.9977392e-01 9.9984455e-01 9.6989787e-01 9.9696547e-01 1.2079746e-02
 9.9948883e-01 9.9959701e-01 9.9981886e-01 9.9911457e-01 9.9907070e-01
 9.9097532e-01 9.9660885e-01 9.9905318e-01 3.0842133e-02 9.9952996e-01
 8.4116182e-04 9.9491173e-01 9.9976176e-01 1.7061342e-03 9.8084199e-01
 9.9741495e-01 9.9940205e-01 9.9954176e-01 9.6899688e-01 9.9608648e-01
 9.9422014e-01 9.9902213e-01 9.9927622e-01 9.9840134e-01 1.2193220e-02
 9.9983156e-01 9.9863356e-01 9.9444121e-01 9.9842453e-01 4.5052073e-03
 9.9778807e-